In [22]:
from rdkit.Chem import Descriptors, MolFromSmiles, MolToSmiles
from rdkit import Chem

In [1]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

from rdkit.Chem import Descriptors, MolFromSmiles

from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

dir1 = '/rds-d2/user/wjm41/hpc-work/datasets/ZINC/real/'

dir2 = '/rds-d7/project/rds-ZNFRY9wKoeE/EnamineREAL/'

scored_dirs1 = open(dir1+'scored_dirs.txt', 'r').read().splitlines()
scored_dirs2 = open(dir2+'scored_dirs.txt', 'r').read().splitlines()

# dirs1 = [x for x in os.listdir(dir1) if os.path.isdir(dir1+x)]
# 
# dirs2 = [x for x in os.listdir(dir2) if os.path.isdir(dir2+x)]

def proc_df(path, N=1000):
    df = pd.read_csv(path)
    df = df[df['2body_score'].notnull()]
    if len(df)!=0:
        df = df.apply(filter_small, axis=1)
        df = df[df['keep']]
        df = df.sort_values(by='2body_score')
        df = df.iloc[:min(N,len(df))]
    return df

In [6]:
# Find Top-N

N=10000

df_all = []
for i,folder in tqdm(enumerate(scored_dirs1), total = len(scored_dirs1)):
        if os.path.isfile(dir1+folder+'/topN_mac.csv'):
            df = pd.read_csv(dir1+folder+'/topN_mac.csv')
            df_all.append(df)
        
df_all = pd.concat(df_all).sort_values(by='2body_score').drop_duplicates(subset=['smiles'], keep='first')

df_all_2 = []
for i,folder in tqdm(enumerate(scored_dirs2), total = len(scored_dirs2)):
        if os.path.isfile(dir2+folder+'/topN_mac.csv'):
            df = pd.read_csv(dir2+folder+'/topN_mac.csv')
            df_all_2.append(df)
        
df_all_2 = pd.concat(df_all_2).sort_values(by='2body_score').drop_duplicates(subset=['smiles'], keep='first')

df_tot = pd.concat([df_all, df_all_2]).sort_values(by='2body_score').drop_duplicates(subset=['smiles'], keep='first')
df_tot.iloc[:min(N, len(df_all_2))].to_csv(dir2+'topN_final_mac.csv', index=False)
print(len(df_tot))

df_all.iloc[:min(N, len(df_all))].to_csv(dir1+'topN_mac.csv', index=False)

df_all_2.iloc[:min(N, len(df_all_2))].to_csv(dir2+'topN_mac.csv', index=False)

In [7]:
print(len(df_tot.drop_duplicates(subset=['smiles'], keep='first')))

In [2]:
import mols2grid

N = 10000
# df1 = df1.rename(columns={'smiles':'SMILES'})
# df2 = df2.rename(columns={'smiles':'SMILES'})

mols2grid.display(pd.read_csv(dir2+'topN_final_mac.csv').iloc[:N].rename(columns={'smiles':'SMILES'}), template="pages", n_rows=10, n_cols=5, subset=["img"], tooltip=['SMILES', '2body_score'])
# mols2grid.display(df2, template="pages", n_rows=10, n_cols=5, subset=["img"], tooltip=['SMILES'])

In [2]:
df1 = pd.read_csv(dir2+'topN_final_acc_true.csv')
df2 = pd.read_csv('~/topN_final_acc.csv')
# print(len(df1.drop_duplicates(subset=['smiles'], keep='first')))
print(df1.equals(df2))
smiles_x = df1['smiles'].values
smiles_y = df2['smiles'].values

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

print(len(intersection(smiles_x, smiles_y)))
print(len(df1[df1['dir'].str.contains('rds-d2')]))
print(len(df2[df2['dir'].str.contains('rds-d2')]))

df_overlap = df1.merge(df2, how='inner', on='smiles')
# print(len(df_overlap[df_overlap.duplicated(subset='smiles')]))
# print(len(df_overlap))
print(len(df_overlap[df_overlap['dir_x'].str.contains('rds-d2')]))
print(len(df_overlap[df_overlap['dir_x'].str.contains('rds-d7')]))

In [48]:
print(df1.drop_duplicates(subset=['smiles'], keep='first'))
df1[['smiles','2body_score']].to_csv('~/topN_mpro.csv', index=False)

In [35]:
submitted_mols = [
    'Cc1ccc(NCc2nnc3n(C)c(=O)c4ccccc4n23)cc1C',
    'O=C(Nc1ccc2c(=O)cc[nH]c2c1)c1ccc2ccncc2c1',
    'Cc1cn2ccc(C(=O)Nc3nc(-c4cnn(C)c4)cs3)cc2n1',
    'O=C(Cc1c[nH]c2cc(Cl)ccc12)Nc1cccc2[nH]c(=O)[nH]c12',
    'Cc1nnc(CNc2ccc3nnc(-c4ccccc4)n3n2)o1',
    'CCn1c(CNc2nnc(-c3nccn3C)o2)nc2ccccc21',
    'CN(Cc1nnc2ccccn12)C(=O)c1ccnc2ccccc12',
    'Cc1cc(N2CCC(C)N(Cc3ccccc3)CC2)n2ncnc2n1',
    'O=C(Nc1nn2cnnc2s1)c1csc(-c2ccccc2)n1',
    'Cc1cn2c(CNC(=O)c3cnn4ccccc34)c(C)nc2s1',
    'N#CCc1cccc(NC(=O)c2cnn3cc(Br)cnc23)n1',
    'Cc1ccc(Br)nc1C(=O)NC(C)c1nnc2c(=O)[nH]ccn12',
    'O=C(NNc1nc(Br)cn2ccnc12)c1nccn2ccnc12',
    'O=C(NNc1cc(O)nc2ncccc12)c1ccc2[nH]ccc2c1',
    'Cc1cc(C(=O)Nc2cnc3nccn3c2)nn1-c1ccc(F)cc1',
    'Cn1ccn2c(CNC(=O)c3ccc4ncsc4c3)nnc2c1=O',
    'Cn1c(NC(=O)Cc2csc3nccn23)nc2ccc(F)cc21',
    'O=C(N/N=c1\cc[nH]c2ccccc12)c1ccc2ncccc2c1',
    'Cc1cc(O)c2cc(NC(=O)c3cnn4ccc(C)nc34)ccc2n1',
    'Cc1cccc2ncnc(NC(C)c3ccc4[nH]c(=O)[nH]c4c3)c12',
    'Cc1[nH]c2ccccc2c1C(=O)NNc1nc2cccnc2s1',
    'O=C(Nc1cnc2nccn2c1)c1cn2c(Br)cnc2s1',
    'O=C(Nc1cccc2nccnc12)c1c[nH]c2cc([N+](=O)[O-])ccc12',
    'Cc1ccc(C(=O)Nc2c(-c3cnn(C)c3)nc3ccccn23)cn1',
    'c1ccc2oc(C3CCCN(Cc4cnc5cnccn45)C3)nc2c1',
    'Cn1cnnc1-c1cccc(NS(=O)(=O)c2c(Cl)nc3sccn23)c1',
    'Cn1c(=O)oc2cc(S(=O)(=O)Nc3ccc4ncccc4c3)ccc21',
    'CC(NC(=O)c1c(Cl)nc2ccccn12)c1nnc2ccccn12',
    'CC(NC(=O)c1nccc2cccnc12)c1ccc2[nH]c(=O)oc2c1',
    'Cc1noc2ncc(CNc3ccc(-c4nnnn4C)cc3)cc12',
    'CC(NC(=O)c1cnc2sccn2c1=O)c1nc2ccccc2n1C',
    'c1cc(CN2CCCC(c3nnc4ccccn34)C2)n2ccnc2c1',
    'Cc1nc2sccn2c1CN(C)C(=O)c1cn2ccccc2n1',
    'CN(Cc1nc2ccccc2s1)C(=O)c1ccc2c(c1)nnn2C',
    'CC(Cc1nc2ccccc2s1)Nc1ccc2nnnn2n1',
    'Cc1csc2ncc(C(=O)Nc3cccc4ncccc34)c(=O)n12',
    'O=C(NNc1nc2ccccc2[nH]1)c1cc(O)nc2ccccc12',
    'O=C(Nc1cccc2cc[nH]c12)c1cccc2[nH]c(=O)oc12',
    'Cn1c(NC(=O)c2cccnc2-n2cccn2)nc2cc(F)ccc21',
    'Cc1nn2c(CNc3ccc(Cn4cncn4)cc3)c(C)nc2s1',
    'CC(CNC(=O)c1cnc2ccccn12)Nc1nc2ccccc2o1',
    'Cc1nc2cc(C(=O)NNc3nc4ccncc4s3)ccc2o1',
    'Cc1nn(C)c2nc(Cl)cc(NNC(=O)c3ccnc4[nH]nnc34)c12',
    'COc1ccc(CNCCc2cn3ccsc3n2)cc1OC(F)F',
    'CC(NC(=O)c1ccc2oc(=S)[nH]c2c1)c1nnc2ccccn12',
]

def return_canon(smi):
    mol = MolFromSmiles(smi)
    Chem.RemoveStereochemistry(mol)
    return MolToSmiles(mol)

submitted_mols = [return_canon(smi) for smi in submitted_mols]
df_sub = pd.DataFrame(submitted_mols, columns=['smiles'])
df2['cansmiles'] = [return_canon(smi) for smi in df2['smiles']]
df_sub = df2[df2['cansmiles'].isin(submitted_mols)].drop_duplicates(subset=['cansmiles'], keep='first')

# print(df_sub[~df_sub['smiles'].isin(df2['cansmiles'])].drop_duplicates(subset=['smiles'], keep='first'))
# print(len(df2[df2['cansmiles'].isin(submitted_mols)].drop_duplicates(subset=['cansmiles'], keep='first')))

In [37]:
print(df_sub)

In [77]:
print(df_sub['2body_score'].values)
print(actual_scores_sub)

In [89]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100
%matplotlib inline

# actual_scores = []
# # for i,row in tqdm(df2[df2['dir'].str.contains('rds-d2')].iterrows(), total=len(df2[df2['dir'].str.contains('rds-d2')])):
# for i,row in tqdm(df2.iterrows(), total=len(df2)):
#     smi = row['smiles']
#     path = row['dir']
#     df = pd.read_csv(path+'/scores_acc.csv')
#     actual_score = df[df['smiles']==smi]['2body_score'].values[0]
#     actual_scores.append(actual_score)

# actual_scores = np.array(actual_scores)
# # print(actual_scores)
# _, bins, _ = plt.hist(actual_scores[actual_scores<10], bins=30, alpha=0.7, histtype='stepfilled', 
#          label='Including Don-Acc')
# plt.hist(df2['2body_score'].values, bins=bins, alpha=0.7, histtype='stepfilled', 
#           label='Submitted Scores')
         
    
# actual_scores_sub = []
n_wrong = 0
for i,row in tqdm(df_sub.iterrows(), total=len(df_sub)):
    smi = row['smiles']
    path = row['dir']
    ind = row['ind']
    df = pd.read_csv(path+'/scores_acc.csv')
    actual_score = df[df['smiles']==smi]['2body_score'].values[0]
    if actual_score != row['2body_score']:
        print(path)
        print(smi)
        print(actual_score)
        print(row['2body_score'])
        n_wrong+=1
print(n_wrong)
#     actual_scores_sub.append(actual_score)
    
# actual_scores_sub = np.array(actual_scores_sub)
# _, bins, _ = plt.hist(actual_scores_sub, bins=10, alpha=0.7, histtype='stepfilled', 
#          label='Including Don-Acc')
# print(bins)
# print(actual_scores_sub)
# plt.hist(df_sub['2body_score'].values, bins=bins, alpha=0.7, histtype='stepfilled', 
#           label='Submitted Scores')    
    
# plt.hist([x, y], density=True, bins=bins) # alternating

plt.title('top-N Molecules')
# plt.title('Submitted Molecules')

plt.xlabel('2body_score')
plt.ylabel('Frequency')
plt.legend(loc='upper right')
# plt.xlim(right=10)
plt.show()
# print(len(df_sub['2body_score'].values))
# print(len(actual_scores_sub[actual_scores_sub<10]))

In [34]:
from rdkit import DataStructs
from rdkit.Chem import AllChem

missing_mols = ['Cc1[nH]c2ccccc2c1C(=O)NNc1nc2cccnc2s1', 'O=C(NNc1nc2ccccc2[nH]1)c1cc(O)nc2ccccc12']

fp_missing = [AllChem.GetMorganFingerprintAsBitVect(MolFromSmiles(smi), radius=3, nBits=1024) for smi in missing_mols]
fps = [AllChem.GetMorganFingerprintAsBitVect(MolFromSmiles(smi), radius=3, nBits=1024) for smi in df2['cansmiles']]

threshold = 0.5

for i in range(len(fp_missing)):
    print('\n{}'.format(missing_mols[i]))
    for j in range(len(fps)):
        sim = DataStructs.FingerprintSimilarity(fp_missing[i], fps[j])
        if sim>threshold:
            print(df2.iloc[j]['cansmiles'])

In [ ]:
import pickle 

dirs_to_dl = df1['dir'].value_counts()
rescore_dirs = []
for path in tqdm(dirs_to_dl.index):
        pairs = [file for file in os.listdir(path) if "pairs_mpi_" in file]
        if len(pairs)!=0:
            print(len(pairs))
        smi_len = len(open(path+'/mols.smi','r').read().splitlines())
        pickle_len = len(pickle.load(open(path+'/pairs.pickle','rb')))
        
        if smi_len != pickle_len:
#             print(path)
#             print(smi_len, pickle_len)
            rescore_dirs.append(path)
            
print('Num bad dirs: {}'.format(len(rescore_dirs)))

print(len(df1))
print(len(df1[df1['dir'].str.contains('rds-d2')]))
print(len(df1[df1['dir'].str.contains('rds-d7')]))

# good_dirs = 
# score_dirs = df_tot.groupby('dir')

print(len(df1[~df1['dir'].isin(rescore_dirs)]))

In [ ]:
dirs_to_dl = df2['dir'].value_counts()
rescore_dirs = []
for path in tqdm(dirs_to_dl.index):
        pairs = [file for file in os.listdir(path) if "pairs_mpi_" in file]
        if len(pairs)!=0:
            print(len(pairs))
        smi_len = len(open(path+'/mols.smi','r').read().splitlines())
        pickle_len = len(pickle.load(open(path+'/pairs.pickle','rb')))
        
        if smi_len != pickle_len:
#             print(path)
#             print(smi_len, pickle_len)
            rescore_dirs.append(path)
            
print('Num bad dirs: {}'.format(len(rescore_dirs)))

print(len(df2))
print(len(df2[df2['dir'].str.contains('rds-d2')]))
print(len(df2[df2['dir'].str.contains('rds-d7')]))

# good_dirs = 
# score_dirs = df_tot.groupby('dir')

print(len(df2[~df2['dir'].isin(rescore_dirs)]))

In [12]:
dirs_to_dl = df_tot['dir'].value_counts()
print(len(dirs_to_dl))


tranches = open(dir+'/all_tranches.txt','r').read().splitlines()
tranches_to_dl = []
for path in dirs_to_dl.index:
    fold = path[-6:]
    for tranch in tranches:
        if fold in tranch:
            tranches_to_dl.append(tranch)
            
print(len(tranches_to_dl))

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(dirs_to_dl)

In [26]:
tranches = open(dir+'/all_tranches.txt','r').read().splitlines()
tranches_to_dl = []
for path in rescore_dirs:
    fold = path[-6:]
    for tranch in tranches:
        if fold in tranch:
            tranches_to_dl.append(fold)

f = open('/rds-d7/project/rds-ZNFRY9wKoeE/val_dir/rescore_folds.txt', 'w')
for line in tranches_to_dl:
    f.write(line+'\n')
f.close()

In [ ]:
# import mols2grid
# 
# df1 = df1.rename(columns={'smiles':'SMILES'})
# df2 = df2.rename(columns={'smiles':'SMILES'})

# mols2grid.display(df1, template="pages", n_rows=10, n_cols=5, subset=["img"], tooltip=['SMILES'])
mols2grid.display(df2, template="pages", n_rows=10, n_cols=5, subset=["img"], tooltip=['SMILES', '2body_score'])

In [ ]:
import mols2grid

# print(df_tot)
# df_tot = pd.concat([df_all, df_all_2]).sort_values(by='2body_score')
# df_tot = df_tot.iloc[:min(N, len(df_all_2))]
# df_tot = df_tot.rename(columns={'smiles':'SMILES'})
# df_all = df_all.rename(columns={'smiles':'SMILES'})
# df_all_2 = df_all_2.rename(columns={'smiles':'SMILES'})
# mols2grid.display(df_all, nrows=6, subset=["img"], tooltip=['SMILES'])
# mols2grid.display(df_all, template="table", n_cols=5, subset=["img"], tooltip=['SMILES'])
# mols2grid.display(df_all, template="pages", n_rows=10, n_cols=5, subset=["img"], tooltip=['SMILES'])
# mols2grid.display(df_all_2, template="pages", n_rows=10, n_cols=5, subset=["img"], tooltip=['SMILES'])
mols2grid.display(df_all.rename(columns={'smiles':'SMILES'}), template="pages", n_rows=10, n_cols=5, subset=["img"], tooltip=['SMILES'])
# page = mols2grid.to_pages(df_all, tooltip=['SMILES'], nrows=6)

In [ ]:
mols2grid.save(df_tot, template="pages", n_rows=10, n_cols=5, subset=["img"], tooltip=['SMILES'], 
               output=dir2+'topN_grid_acc.html')

# Validate previous top-N

In [1]:
import pandas as pd

df = pd.read_csv('~/topN_final_acc.csv')
print(df)